In [ ]:
%%script false --no-raise-error
# run this to install things that aren't on colab by default! (comment out the %% line above)
!pip install preprocessor
!pip install auto-sklearn==0.14.3

In [ ]:
import autosklearn.classification
import autosklearn.metrics
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import preprocessor as p

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

data loading functions

In [ ]:
basepath = "/content/drive/MyDrive/DataScienceMaster/Experiment Design/"

# set this true if you want to actually run the AutoML section!
run = False

In [ ]:
twitter_dict = {'racism':1,'sexism':2,'none':0}
def load_data(filename):
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        label = data[i]['label']
        if N_CLASS == 3:
          label = twitter_dict[label]
        labels.append(label)
    return x_text,labels

def get_filename(dataset):
    global N_CLASS, HASH_REMOVE
    if(dataset=="twitter"):
        filename = basepath+"twitter_data.pkl"
        N_CLASS = 3
        HASH_REMOVE = False
    elif(dataset=="formspring"):
        N_CLASS = 2
        filename = basepath+"formspring_data.pkl"
        HASH_REMOVE = False
    elif(dataset=="wiki"):
        N_CLASS = 2
        filename = basepath+"wiki_data.pkl"
        HASH_REMOVE = False
    return filename


In [ ]:
def get_train_test_data(data, labels, datarep="char", split=True):
  if datarep == "char":
    print("Using char n-grams based features")
    bow_transformer = CountVectorizer(max_features = 10000, ngram_range = (1,2)).fit(data)
    comments_bow = bow_transformer.transform(data)
    tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
    comments_tfidf = tfidf_transformer.transform(comments_bow)
    features = comments_tfidf
  elif datarep == "word":
    print("Using word based features")
    bow_transformer = CountVectorizer(analyzer="word",max_features = 10000,stop_words='english').fit(data)
    comments_bow = bow_transformer.transform(data)
    tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
    comments_tfidf = tfidf_transformer.transform(comments_bow)
    features = comments_tfidf
  else:
    print("invalid data represenation! Must be either 'char' or 'word'.")
    raise ValueError

  if split:
    train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.2, random_state=31415, shuffle=True)
    return train_x, train_y, test_x, test_y
  else:
    return features, labels

def eval_model(model, x_test, y_test):
  y_pred = model.predict(x_test)
  y_test = np.array([int(x) for x in y_test])
  y_pred = np.array([int(x) for x in y_pred])
  return np.array([f1_score(y_test, y_pred, average=None)[1:]])


In [ ]:
# do auto-ml!

def get_model(train_x, train_y, time_limit=20*60): 
  automl = autosklearn.classification.AutoSklearnClassifier(
      time_left_for_this_task = time_limit,
      per_run_time_limit = time_limit//4,
      memory_limit = 6000,
      metric=autosklearn.metrics.f1_macro,
      exclude = {'classifier':["mlp"]},
      ensemble_size = 1,
      n_jobs = -1      
  )
  automl.fit(train_x, train_y)
  return automl

In [ ]:
results_char = {}

In [ ]:
if run:
  for dataset in ["formspring", "twitter", "wiki"]:
    data, labels = load_data(get_filename(dataset))
    print("got {} data".format(dataset))
    x_train, y_train, x_test, y_test = get_train_test_data(data, labels, datarep="char")
    model = get_model(x_train, y_train)
    results = eval_model(model, x_test, y_test)
    results_char[dataset] = {"scores":results, "model":model}


In [ ]:
if run:
  with open(basepath+"results_tradml_charrepr.pkl", "wb") as f:
    pickle.dump(results_char, f, pickle.HIGHEST_PROTOCOL)
else:
  results_char = pickle.load(open(basepath+"results_tradml_charrepr.pkl", "rb"))

In [ ]:
results_word = {}

In [ ]:
if run:
  for dataset in ["formspring", "twitter", "wiki"]:
    data, labels = load_data(get_filename(dataset))
    print("got {} data".format(dataset))
    x_train, y_train, x_test, y_test = get_train_test_data(data, labels, datarep="word")
    model = get_model(x_train, y_train)
    results = eval_model(model, x_test, y_test)
    results_word[dataset] = {"scores":results, "model":model}

In [ ]:
if run:
  with open(basepath+"results_tradml_wordrepr.pkl","wb") as f:
    pickle.dump(results_word, f, pickle.HIGHEST_PROTOCOL)
else:
  results_word = pickle.load(open(basepath+"results_tradml_wordrepr.pkl", "rb"))

In [ ]:
results_char

{'formspring': {'model': AutoSklearnClassifier(ensemble_size=1, exclude={'classifier': ['mlp']},
                        memory_limit=6000, metric=f1_macro, n_jobs=-1,
                        per_run_time_limit=300, time_left_for_this_task=1200),
  'scores': array([[0.48201439],
         [0.41104294],
         [0.44370861]])},
 'twitter': {'model': AutoSklearnClassifier(ensemble_size=1, exclude={'classifier': ['mlp']},
                        memory_limit=6000, metric=f1_macro, n_jobs=-1,
                        per_run_time_limit=300, time_left_for_this_task=1200),
  'scores': array([[0.68287037, 0.7210084 ],
         [0.78042328, 0.69529984],
         [0.72839506, 0.70792079]])},
 'wiki': {'model': AutoSklearnClassifier(ensemble_size=1, exclude={'classifier': ['mlp']},
                        memory_limit=6000, metric=f1_macro, n_jobs=-1,
                        per_run_time_limit=300, time_left_for_this_task=1200),
  'scores': array([[0.5620438 ],
         [0.80297952],
         [0.

In [ ]:
results_word

{'formspring': {'model': AutoSklearnClassifier(ensemble_size=1, exclude={'classifier': ['mlp']},
                        memory_limit=6000, metric=f1_macro, n_jobs=-1,
                        per_run_time_limit=300, time_left_for_this_task=1200),
  'scores': array([[0.35950413],
         [0.53374233],
         [0.42962963]])},
 'twitter': {'model': AutoSklearnClassifier(ensemble_size=1, exclude={'classifier': ['mlp']},
                        memory_limit=6000, metric=f1_macro, n_jobs=-1,
                        per_run_time_limit=300, time_left_for_this_task=1200),
  'scores': array([[0.72922252, 0.81150794],
         [0.71957672, 0.66288493],
         [0.72436751, 0.72970562]])},
 'wiki': {'model': AutoSklearnClassifier(ensemble_size=1, exclude={'classifier': ['mlp']},
                        memory_limit=6000, metric=f1_macro, n_jobs=-1,
                        per_run_time_limit=300, time_left_for_this_task=1200),
  'scores': array([[0.54405941],
         [0.81862197],
         [0.

In [ ]:
for ds in results_char:
  print("char representation")
  f1 = results_char[ds]["scores"][-1]
  print(ds, f1)
print()
for ds in results_word:
  print("word representation")
  f1 = results_word[ds]["scores"][-1]
  print(ds, f1)

char representation
formspring [0.44370861]
char representation
twitter [0.72839506 0.70792079]
char representation
wiki [0.66124827]

word representation
formspring [0.42962963]
word representation
twitter [0.72436751 0.72970562]
word representation
wiki [0.6536803]


In [ ]:
def cv_model(X, y, model):
  X,y = shuffle(X, y, random_state=42)
  kf = KFold(n_splits=10)
  scores = []
  for train_index, test_index in kf.split(X):
    y = np.asarray(y)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.refit(X_train,y_train)
    y_pred = model.predict(X_test)
    curr_scores = eval_model(model, X_test, y_test)
    scores.append(np.hstack(curr_scores))
  return scores

In [ ]:
scoredict = {}

for n,resultsdict in enumerate([results_char, results_word]):
  repr = "none"
  if n == 0:
    repr = "char"
  else:
    repr = "word"
  scoredict[repr] = {}
  for ds in resultsdict:
    orig_f1 = resultsdict[ds]["scores"][-1]
    model = resultsdict[ds]["model"]
    modeltype = model.leaderboard()["type"].values[0]
    print("training {} model on {} data, original test set score: {}".format(modeltype, ds, [round(x, 3) for x in orig_f1]))

    data, labels = load_data(get_filename(ds))
    features, labels = get_train_test_data(data, labels, datarep=repr, split=False)
    scores = cv_model(features, labels, model)
    avgscore = np.mean(scores, axis=0)
    print("Average CV score: ",np.round(avgscore,3))
    if ds != "twitter":
      scoredict[repr][ds] = {"model":modeltype, "cv_f1score":avgscore}
    else:
      scoredict[repr][ds] = {"model":modeltype, "cv_f1score_class1":avgscore[0], "cv_f1score_class2":avgscore[1]}
  print()

training extra_trees model on formspring data, original test set score: [0.444]
Using char n-grams based features
Average CV score:  [0.488]
training sgd model on twitter data, original test set score: [0.728, 0.708]
Using char n-grams based features
Average CV score:  [0.731 0.724]
training sgd model on wiki data, original test set score: [0.661]
Using char n-grams based features
Average CV score:  [0.662]

training extra_trees model on formspring data, original test set score: [0.43]
Using word based features
Average CV score:  [0.454]
training random_forest model on twitter data, original test set score: [0.724, 0.73]
Using word based features
Average CV score:  [0.756 0.739]
training sgd model on wiki data, original test set score: [0.654]
Using word based features
Average CV score:  [0.654]



In [ ]:
pickle.dump(scoredict, open(basepath+"cv_results.pkl", "wb"))